<a href="https://colab.research.google.com/github/Alymostafa/PRML/blob/master/AllDataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##weights and biases

In [ ]:
!pip install --upgrade wandb

In [ ]:
!wandb login d3214840db57c8ceb248f2168774d9d01ddd1d3c

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##imports

In [157]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import metrics
from pprint import pprint
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re 
from nltk.corpus import stopwords
#from sklearn.datasets import fetch_rcv1


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##<h1>First DataSets</h1>

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
def twenty_newsgroup_to_csv(data,name):
    newsgroups_train = fetch_20newsgroups(subset= data , remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    out.to_csv(name+'.csv')

twenty_newsgroup_to_csv('train','train')
twenty_newsgroup_to_csv('test','test')

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df.dropna(subset = ["text"], inplace=True)
df_test.dropna(subset = ["text"], inplace=True)

In [ ]:
all_names = names.words()
WNL = WordNetLemmatizer()
def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1 
    return(list(cleaned.values()))

In [ ]:
df['text'] = clean(df['text'])
df_test['text'] = clean(df_test['text'])

In [ ]:
print(len(df))
print(len(df_test))


11096
7370


In [ ]:
df.drop(labels=['target','date'],axis=1,inplace=True)


In [ ]:
df_test.drop(labels=['target','date'],axis=1,inplace=True)


##<h1>Second Dataset</h1>

In [ ]:
nltk.download('reuters')
nltk.download('punkt')
from nltk.corpus import stopwords, reuters

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
documents = reuters.fileids()
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))
 
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

In [ ]:
docs_pdf_train = pd.DataFrame(train_docs) 
docs_pdf_test = pd.DataFrame(test_docs) 
docs_pdf_train['text'] = pd.DataFrame(train_docs) 
docs_pdf_test['text'] = pd.DataFrame(test_docs) 
docs_pdf_train.drop([docs_pdf_train.columns[0]], axis ='columns',inplace=True)
docs_pdf_test.drop([docs_pdf_test.columns[0]], axis ='columns',inplace=True)
docs_pdf_train.dropna(inplace=True)
docs_pdf_test.dropna(inplace=True)

In [ ]:
docs_pdf_train.dropna(subset = ["text"], inplace=True)
docs_pdf_train.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'   ', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'   ', ' ', regex=True,inplace=True)

In [ ]:
docs_pdf_train['text'] = clean(docs_pdf_train['text'])
docs_pdf_test['text'] = clean(docs_pdf_test['text'])

In [ ]:
train_labels = [reuters.categories(doc_id)for doc_id in train_docs_id]
test_labels = [reuters.categories(doc_id)for doc_id in test_docs_id]

In [153]:
achedStopWords = stopwords.words("english")
 
def tokenize(text):
  min_length = 3
  words = map(lambda word: word.lower(), word_tokenize(text))
  words = [word for word in words if word not in achedStopWords]
  tokens = (list(map(lambda token: PorterStemmer().stem(token),
                                   words)))
  p = re.compile('[a-zA-Z]+');
  filtered_tokens =list(filter (lambda token: p.match(token) and
                               len(token) >= min_length,
                               tokens))
  return filtered_tokens

def word_tokenizer():
  tokenize_docs = []
  docs_tokens = docs_pdf_train['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs = word_tokenizer()

In [ ]:
def word_tokenizer_test():
  tokenize_docs = []
  docs_tokens = docs_pdf_test['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs_test = word_tokenizer_test()

In [ ]:
df_train_2=pd.DataFrame(docs_pdf_train,columns=['text'])
df_train_2['target']=''
df_train_2['target']=train_labels


In [ ]:
df_test_2=pd.DataFrame(docs_pdf_test,columns=['text'])
df_test_2['target']=''
df_test_2['target']=test_labels
df_test_2

,text,target
0,asian exporter fear damage from japan rift mou...,[trade]
1,china daily say vermin eat pct grain stock a s...,[grain]
2,japan to revise long term energy demand downwa...,"[crude, nat-gas]"
3,thai trade deficit widens in first quarter tra...,"[corn, grain, rice, rubber, sugar, tin, trade]"
4,indonesia see cpo price rising sharply indones...,"[palm-oil, veg-oil]"
...,...,...
3014,chase corp make offer for entregrowth corp ltd...,[acq]
3015,tokyo dealer see dollar poised to breach yen f...,"[dlr, money-fx, yen]"
3016,conference cut gulf war risk charge the pakist...,[ship]
3017,soviet industrial slower in the soviet industr...,[ipi]


In [ ]:
df_train_2.rename(columns={'target':'title'},inplace=True)
df_test_2.rename(columns={'target':'title'},inplace=True)


In [ ]:
frames_train=[df,df_train_2]
frames_test=[df_test,df_test_2]
all_dataframes_train=pd.concat(frames_train,keys=['x','y'])
all_dataframes_test=pd.concat(frames_test,keys=['x','y'])


In [ ]:
all_dataframes_train.drop(labels='Unnamed: 0',inplace=True,axis=1)
all_dataframes_test.drop(labels='Unnamed: 0',inplace=True,axis=1)

##Bert

In [ ]:
#import wandb
#from wandb.keras import WandbCallback
#wandb.init(project="gru-wikipedia")

In [1]:
!pip install 'kashgari>=2.0.0'	

In [238]:
import logging
from kashgari.embeddings import BertEmbedding
from kashgari.tasks.classification import BiGRU_Model
from kashgari.tasks.classification import CNN_LSTM_Model


In [239]:
logging.basicConfig(level='DEBUG')


In [4]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-10-10 13:33:54--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   183MB/s    in 2.1s    

2020-10-10 13:33:57 (183 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [5]:
!unzip '/content/uncased_L-12_H-768_A-12.zip'

Archive:  /content/uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [240]:
bert_embed = BertEmbedding('/content/uncased_L-12_H-768_A-12')

2020-10-10 16:40:50,816 [DEBUG] kashgari - ------------------------------------------------
2020-10-10 16:40:50,817 [DEBUG] kashgari - Loaded transformer model's vocab
2020-10-10 16:40:50,818 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-10-10 16:40:50,818 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-10-10 16:40:50,819 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-10-10 16:40:50,820 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

In [252]:
model = CNN_LSTM_Model(bert_embed)

In [ ]:
;

In [6]:
toknize_train=all_dataframes_train['text'].tolist()
toknize_train=[i.split(' ') for i in toknize_train]

NameError: ignored

In [7]:
modifed_list_labels=all_dataframes_train['title'][:11096].tolist()
modifed_list_labels=[[i] for i in modifed_list_labels]


NameError: ignored

In [ ]:
all_dataframes_train['title'][:11096]=modifed_list_labels
train_labels_=all_dataframes_train['title'].tolist()


In [ ]:
toknize_test=all_dataframes_test['text'].tolist()
toknize_test=[i.split(' ') for i in toknize_test]


In [ ]:
modifed_list_labels_test=all_dataframes_test['title'][:7370].tolist()
modifed_list_labels_test=[[i] for i in modifed_list_labels_test]
all_dataframes_test['title'][:7370]=modifed_list_labels_test
test_labels_=all_dataframes_test['title'].tolist()


##Wikipedia Dataset

In [8]:
!wget 'https://www.kaggleusercontent.com/kf/44384181/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..QOzonC0Hx7LMCzRHKFu4tg.6Fm8mESdwaUzIItZorn3YFcBAYyU1F0ha5fRf8EAYOtJjjuadBbYFyfs1JuFWer0WyLY4d-7Srx4XUoTXkTGbAgGgdHdS4CAjMm2JdGGXaVcvP8vIv6qf64m55AXjmGDtexR9UMQN84E5oU4RyZLQjdJdUpCe47WlIAKmPEspbfit27g-qkPx-hAFcYB5peDveSUhB8n5Pv9Rekgff6lseRsNID7o2FLEpIM__bZVpuGMSgrJhHTcXg_sKHJSkR1A0h2sxonBvnc56uWted_JGfFGrI238BtuHpq8CwCrE1dJzemcYIeiUtkXgoYvWRpSwhLiss1Lt0iR8-4wt9vHYM5n3Xvf0KMFBtY182hIxgEYlDhJ7rHDoyFUS_tIC9AGk69jay_xRK59jx74ilkLabqXmNg6wqC9Iuhi_h_T-5UnYt_RQnqWTvc9oZZ-nkNx9YOwTd0bvibIHyOMGlTxGoZSKovhoBaUEiLOIOjt-0wuRDgpbvDwf1FRJqBpaw1oEVqDE1NQnerTda2TPwuZRvi1YHngOVxbGQwzbc1pOfNQrJxaw35CXLkzjZCaM2IgYj38Os-gx3EHmVNgaWzLdlXwyc7cYHnUCxkJpeJVGJw1ki5v0ZbitXeWObLEZmi.rr539V7R_WKziBL-XMIt2A/all_dataset_fin.csv'

--2020-10-10 13:34:08--  https://www.kaggleusercontent.com/kf/44384181/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..QOzonC0Hx7LMCzRHKFu4tg.6Fm8mESdwaUzIItZorn3YFcBAYyU1F0ha5fRf8EAYOtJjjuadBbYFyfs1JuFWer0WyLY4d-7Srx4XUoTXkTGbAgGgdHdS4CAjMm2JdGGXaVcvP8vIv6qf64m55AXjmGDtexR9UMQN84E5oU4RyZLQjdJdUpCe47WlIAKmPEspbfit27g-qkPx-hAFcYB5peDveSUhB8n5Pv9Rekgff6lseRsNID7o2FLEpIM__bZVpuGMSgrJhHTcXg_sKHJSkR1A0h2sxonBvnc56uWted_JGfFGrI238BtuHpq8CwCrE1dJzemcYIeiUtkXgoYvWRpSwhLiss1Lt0iR8-4wt9vHYM5n3Xvf0KMFBtY182hIxgEYlDhJ7rHDoyFUS_tIC9AGk69jay_xRK59jx74ilkLabqXmNg6wqC9Iuhi_h_T-5UnYt_RQnqWTvc9oZZ-nkNx9YOwTd0bvibIHyOMGlTxGoZSKovhoBaUEiLOIOjt-0wuRDgpbvDwf1FRJqBpaw1oEVqDE1NQnerTda2TPwuZRvi1YHngOVxbGQwzbc1pOfNQrJxaw35CXLkzjZCaM2IgYj38Os-gx3EHmVNgaWzLdlXwyc7cYHnUCxkJpeJVGJw1ki5v0ZbitXeWObLEZmi.rr539V7R_WKziBL-XMIt2A/all_dataset_fin.csv
Resolving www.kaggleusercontent.com (www.kaggleusercontent.com)... 35.190.26.106
Connecting to www.kaggleusercontent.com (www.kaggleusercontent.com)|35.190.26.106|:443... co

In [8]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
df=pd.read_csv("/content/all_dataset_fin.csv")

In [137]:
df[df['title']=='Miscellaneous']

,Unnamed: 0,title,text
113,113,Miscellaneous,"['a', 'government', 'is', 'the', 'system', 'to..."
114,114,Miscellaneous,"['fighters', 'and', 'a', 'tanker', 'aircraft',..."
115,115,Miscellaneous,"['allies', 'day', 'may', 'national', 'gallery'..."
116,116,Miscellaneous,"['the', 'word', 'authority', 'derived', 'from'..."
117,117,Miscellaneous,"['bureau', 'may', 'refer', 'to', 'public', 'ad..."
118,118,Miscellaneous,"['a', 'council', 'is', 'a', 'group', 'of', 'pe..."
119,119,Miscellaneous,"['the', 'sun', 'is', 'the', 'source', 'of', 'e..."
120,120,Miscellaneous,"['environment', 'may', 'refer', 'to', 'environ..."
121,121,Miscellaneous,"['the', 'food', 'and', 'drug', 'administration..."
122,122,Miscellaneous,"['an', 'institute', 'is', 'an', 'organisationa..."


In [101]:
df['title'].unique()

array(['Academic_Science', 'Business', 'Community', 'Computing',
       'Governmental', 'Miscellaneous', 'Medical'], dtype=object)

In [95]:
Academic_Science = df[df['title']=='Academic_Science']
Business = df[df['title']=='Business']
Community = df[df['title']=='Community']
Computing = df[df['title']=='Computing']
Governmental = df[df['title']=='Governmental']
Miscellaneous = df[df['title']=='Miscellaneous']
Medical = df[df['title']=='Medical']

In [96]:
list_Academic_Science = list(Academic_Science['text'])
list_Business = list(Business['text'])
list_Community = list(Community['text'])
list_Computing = list(Computing['text'])
list_Governmental = list(Governmental['text'])
list_Miscellaneous = list(Miscellaneous['text'])
list_Medical = list(Medical['text'])

In [97]:
import ast
list_Academic_Science=[ast.literal_eval(i) for i in list_Academic_Science]
list_Business=[ast.literal_eval(i) for i in list_Business]
list_Community=[ast.literal_eval(i) for i in list_Community]
list_Computing=[ast.literal_eval(i) for i in list_Computing]
list_Governmental=[ast.literal_eval(i) for i in list_Governmental]
list_Miscellaneous=[ast.literal_eval(i) for i in list_Miscellaneous]
list_Medical=[ast.literal_eval(i) for i in list_Medical]

In [98]:
count = 0
list_list_Aca=[]
list_Aca= [] 
for w in list_Academic_Science:
  for j in w:
    list_Aca.append(j)
    count = count + 1
    if count == 40:
      list_list_Aca.append(list_Aca)
      count = 0
      list_Aca = []


count = 0
list_list_bus=[]
list_bus= [] 
for w in list_Business:
  for j in w:
    list_bus.append(j)
    count = count + 1
    if count == 40:
      list_list_bus.append(list_bus)
      count = 0
      list_bus = []


count = 0
list_list_comm=[]
list_comm= [] 
for w in list_Community:
  for j in w:
    list_comm.append(j)
    count = count + 1
    if count == 40:
      list_list_comm.append(list_comm)
      count = 0
      list_comm = []


count = 0
list_list_comp=[]
list_comp= [] 
for w in list_Computing:
  for j in w:
    list_comp.append(j)
    count = count + 1
    if count == 40:
      list_list_comp.append(list_comp)
      count = 0
      list_comp = []

count = 0
list_list_cov=[]
list_cov= [] 
for w in list_Governmental:
  for j in w:
    list_cov.append(j)
    count = count + 1
    if count == 40:
      list_list_cov.append(list_cov)
      count = 0
      list_cov = []


count = 0
list_list_mis=[]
list_mis= [] 
for w in list_Miscellaneous:
  for j in w:
    list_mis.append(j)
    count = count + 1
    if count == 40:
      list_list_mis.append(list_mis)
      count = 0
      list_mis = []


count = 0
list_list_med=[]
list_med= [] 
for w in list_Medical:
  for j in w:
    list_med.append(j)
    count = count + 1
    if count == 40:
      list_list_med.append(list_med)
      count = 0
      list_med = []




In [105]:
df_aca = pd.DataFrame()
df_aca['text'] = list_list_Aca
df_aca['title'] = "acadamic_science"

df_bus = pd.DataFrame()
df_bus['text'] = list_list_bus
df_bus['title'] = "Business"

df_comm = pd.DataFrame()
df_comm['text'] = list_list_comm
df_comm['title'] = "community"

df_comp = pd.DataFrame()
df_comp['text'] = list_list_comp
df_comp['title'] = "computing"

df_gov = pd.DataFrame()
df_gov['text'] = list_list_cov
df_gov['title'] = "govermnetal"

df_mis = pd.DataFrame()
df_mis['text'] = list_Miscellaneous
df_mis['title'] = "Miscellaneous"

df_med = pd.DataFrame()
df_med['text'] = list_list_med
df_med['title'] = "medical"

In [133]:
df_all_cat = pd.concat([df_aca,df_bus,df_comm,df_comp,df_gov,df_mis,df_med],ignore_index = True)

In [136]:
df_all_cat[df_all_cat['title']=='Miscellaneous']

,text,title
13422,"[a, government, is, the, system, to, govern, a...",Miscellaneous
13423,"[fighters, and, a, tanker, aircraft, of, the, ...",Miscellaneous
13424,"[allies, day, may, national, gallery, of, art,...",Miscellaneous
13425,"[the, word, authority, derived, from, the, lat...",Miscellaneous
13426,"[bureau, may, refer, to, public, administratio...",Miscellaneous
13427,"[a, council, is, a, group, of, people, who, co...",Miscellaneous
13428,"[the, sun, is, the, source, of, energy, for, m...",Miscellaneous
13429,"[environment, may, refer, to, environment, bio...",Miscellaneous
13430,"[the, food, and, drug, administration, fda, or...",Miscellaneous
13431,"[an, institute, is, an, organisational, body, ...",Miscellaneous


In [112]:
msk = np.random.rand(len(df_all_cat)) < 0.9
train_df = df_all_cat[msk]
test_val_df = df_all_cat[~msk]

In [113]:
msk = np.random.rand(len(test_val_df)) < 0.5
test_df = test_val_df[msk]
validate_df = test_val_df[~msk]

In [ ]:
df[df['title']=='Miscellaneous']

In [123]:
print(f'train data count: {len(train_df)}')
print(f'test data count: {len(test_df)}')
print(f'validate data count: {len(validate_df)}')
train_x, train_y = list(train_df['text']), list(train_df['title'])
test_x, test_y = list(test_df['text']), list(test_df['title'])
validate_x, validate_y = list(validate_df['text']), list(validate_df['title'])

train data count: 15004
test data count: 790
validate data count: 856


In [244]:
X_train_filtered_list=[]
X_train_filtered= [] 
for w in train_x:
  for j in w:
    if j not in stop_words: 
      X_train_filtered.append(j)
  X_train_filtered_list.append(X_train_filtered)

In [247]:
X_test_filtered_list=[]
X_test_filtered= [] 
for w in validate_x:
  for j in w:
    if j not in stop_words: 
      X_test_filtered.append(j)
  X_test_filtered_list.append(X_test_filtered)

In [ ]:
model.fit(X_train_filtered_list,train_y,X_test_filtered_list,validate_y,epochs=50)

In [138]:
model.evaluate(test_x,test_y)

2020-10-10 16:04:10,513 [WARNING] kashgari - Sequence length is None, will use the max embedding seq length, which is 512
2020-10-10 16:04:10,542 [DEBUG] kashgari - predict input shape (2, 790, 512) x: 
(array([[ 101, 1998, 2353, ...,    0,    0,    0],
       [ 101, 2968, 3802, ...,    0,    0,    0],
       [ 101, 1996, 2034, ...,    0,    0,    0],
       ...,
       [ 101,  100, 2542, ...,    0,    0,    0],
       [ 101, 1997, 1996, ...,    0,    0,    0],
       [ 101, 2554, 2001, ...,    0,    0,    0]], dtype=int32), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32))
2020-10-10 16:04:46,655 [DEBUG] kashgari - predict output shape (790, 7)
2020-10-10 16:04:46,658 [DEBUG] kashgari - predict output argmax: [0 3 5 0 1 0 0 1 1 4 0 0 0 1 1 1 0 0 0 0 5 0 0 3 4 5 5 0 4 0 0 0 5 0 2 2 1
 2 5 2 2 0 2 0 0 2 0 2 0 2 2 2 5 5 0

                  precision    recall  f1-score   support

        Business     0.9104    0.7262    0.8079        84
   Miscellaneous     0.0000    0.0000    0.0000         1
acadamic_science     0.9402    0.4029    0.5641       273
       community     0.5952    0.8547    0.7018       117
       computing     0.6176    0.9403    0.7456        67
     govermnetal     0.6744    0.9355    0.7838        93
         medical     0.7087    0.9419    0.8089       155

        accuracy                         0.7177       790
       macro avg     0.6352    0.6859    0.6303       790
    weighted avg     0.7807    0.7177    0.6990       790



{'detail': {'Business': {'f1-score': 0.8079470198675497,
   'precision': 0.9104477611940298,
   'recall': 0.7261904761904762,
   'support': 84},
  'Miscellaneous': {'f1-score': 0.0,
   'precision': 0.0,
   'recall': 0.0,
   'support': 1},
  'acadamic_science': {'f1-score': 0.5641025641025641,
   'precision': 0.9401709401709402,
   'recall': 0.40293040293040294,
   'support': 273},
  'accuracy': 0.7177215189873418,
  'community': {'f1-score': 0.7017543859649124,
   'precision': 0.5952380952380952,
   'recall': 0.8547008547008547,
   'support': 117},
  'computing': {'f1-score': 0.7455621301775149,
   'precision': 0.6176470588235294,
   'recall': 0.9402985074626866,
   'support': 67},
  'govermnetal': {'f1-score': 0.7837837837837838,
   'precision': 0.6744186046511628,
   'recall': 0.9354838709677419,
   'support': 93},
  'macro avg': {'f1-score': 0.6302877356891861,
   'precision': 0.6352371891650611,
   'recall': 0.68593422801759,
   'support': 790},
  'medical': {'f1-score': 0.80886426

In [151]:
def predict(docs):
  tokenize_docs = []
  df=pd.DataFrame([docs],columns=['text'])
  docs_tokens = df['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [235]:
x = predict('Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.[3] Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as learning and problem solving.[4]As machines become increasingly capable, tasks considered to require intelligence are often removed from the definition of AI, a phenomenon known as the AI effect.[5] A quip in Teslers Theorem says "AI is whatever hasnt been done yet.[6] For instance, optical character recognition is frequently excluded from things considered to be AI,[7] having become a routine technology.[8] Modern machine capabilities generally classified as AI include successfully understanding human speech,computing')

In [236]:
len(x[0])

95

In [237]:
model.predict(x)

2020-10-10 16:39:16,540 [DEBUG] kashgari - predict input shape (2, 1, 97) x: 
(array([[  101,   100,   100,   100,  2655,   100,   100,   100,   100,
          100,   100,   100,   100,  4653,  2529,   100,  2599, 16432,
          100,  2492,   100,   100,  4005,   100,   100,   100,  2202,
         2895, 20446,   100,  3112,   100,  3125,   100,  2744,   100,
          100,  2411,  2224,   100,   100,   100, 23150,   100,  3853,
         2529,   100,  2529,  2568,  4553,  3291,   100,   100,   100,
          100,   100,  4708,   100,   100,   100,  2411,   100,   100,
         9575,  2124,  3466,   100,   100,  9872,  2360,   100,   100,
         2589,  2664,   100, 22816,   100,   100,  6976,   100,  2518,
          100,   100,   100,   100,  2715,   100,   100,   100,   100,
          100,  3112,  3305,  2529,  4613,   100,   102]], dtype=int32), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

['medical']

In [32]:
train_df

,Unnamed: 0,title,text
0,0,Academic_Science,"['an', 'academic', 'degree', 'is', 'a', 'quali..."
1,1,Academic_Science,"['an', 'alumnus', 'masculine', 'plural', 'alum..."
2,2,Academic_Science,"['an', 'example', 'of', 'an', 'amateur', 'radi..."
3,3,Academic_Science,"['anthropology', 'is', 'the', 'study', 'of', '..."
4,4,Academic_Science,"['roman', 'ruins', 'lausanne', 'switzerland', ..."
...,...,...,...
154,154,Medical,"['rehabilitation', 'psychology', 'is', 'a', 's..."
155,155,Medical,"['surgeons', 'repairing', 'a', 'ruptured', 'ac..."
156,156,Medical,"['a', 'syndrome', 'is', 'a', 'set', 'of', 'med..."
157,157,Medical,"['therapy', 'often', 'abbreviated', 'tx', 'tx'..."


In [13]:
import ast
X_train=[ast.literal_eval(i) for i in X_train]
X_test=[ast.literal_eval(i) for i in X_test]

In [243]:
stop_words = set(stopwords.words('english')) 


In [ ]:
X_train_filtered_list=[]
X_train_filtered= [] 
for w in X_train:
  for j in w:
    if j not in stop_words: 
      X_train_filtered.append(j)
  X_train_filtered_list.append(X_train_filtered)

In [15]:
X_train_filtered_list=[]
X_train_filtered= [] 
for w in X_train:
  for j in w:
    if j not in stop_words: 
      X_train_filtered.append(j)
  X_train_filtered_list.append(X_train_filtered)

In [16]:
X_test_filtered_list=[]
X_test_filtered= [] 
for w in X_test:
  for j in w:
    if j not in stop_words: 
      X_test_filtered.append(j)
  X_test_filtered_list.append(X_test_filtered)

In [ ]:
X_test_filtered_list[0]

In [125]:
model.fit(X_train_filtered_list,y_train,X_test_filtered_list,y_test,epochs=35,batch_size=20)

NameError: ignored

In [ ]:
model.save("LSTM")

In [ ]:
!mkdir '/content/LSTM' 
!cp  '/content/drive/My Drive/Model_weights/embed_model_weights.h5' '/content/LSTM'
!cp  '/content/drive/My Drive/Model_weights/model_config.json' '/content/LSTM'
!cp  '/content/drive/My Drive/Model_weights/model_weights.h5' '/content/LSTM'


In [ ]:

loaded_model = BiLSTM_Model.load_model('/content/LSTM/')

2020-09-01 17:51:13,866 [DEBUG] kashgari - ------------------------------------------------
2020-09-01 17:51:13,867 [DEBUG] kashgari - Loaded transformer model's vocab
2020-09-01 17:51:13,868 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-09-01 17:51:13,870 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-09-01 17:51:13,871 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-09-01 17:51:13,872 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

In [ ]:
def predict(docs):
  tokenize_docs = []
  df=pd.DataFrame([docs],columns=['text'])
  docs_tokens = df['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
loaded_model.predict([nltk_tokens])

2020-09-01 18:08:44,835 [DEBUG] kashgari - predict input shape (2, 1, 62) x: 
(array([[  101,   100, 13058,   100,  5658,   100, 14931,  1058, 14931,
         5658,  1058,  5096,   100,  1058,   100,   100,   100,   100,
         1058,   100,  3157,   100,   100, 14931,  1058, 14931,  5658,
          100,  1058,   100,  5096,   100,  1058,   100,   100,   100,
          100,  1058,   100,   100,  2566,  3745, 11138,   100,  1997,
         2048,   100,  3745,  1999,  3988,  2270,  3157,   100,  2765,
         2421,  9313,  5114,  1997,   100,  2030, 14931,   102]],
      dtype=int32), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32))
2020-09-01 18:08:44,907 [DEBUG] kashgari - predict output shape (1, 110)


[['earn']]

In [ ]:
loaded_model.evaluate(toknize_test,test_labels_)

2020-09-01 18:10:17,533 [DEBUG] kashgari - predict input shape (2, 10389, 512) x: 
(array([[ 101, 1045, 2572, ...,    0,    0,    0],
       [ 101, 1045, 2031, ...,    0,    0,    0],
       [ 101, 1045, 2342, ...,    0,    0,    0],
       ...,
       [ 101, 3034, 3013, ...,    0,    0,    0],
       [ 101, 3354, 3919, ...,    0,    0,    0],
       [ 101, 2416, 2730, ...,    0,    0,    0]], dtype=int32), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32))
2020-09-01 18:17:21,525 [DEBUG] kashgari - predict output shape (10389, 110)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support
                earn     0.9709    0.9825    0.9767      1087
                 acq     0.9677    0.9583    0.9630       719
      comp.windows.x     0.4296    0.5964    0.4995       394
    rec.sport.hockey     0.9121    0.7143    0.8011       392
soc.religion.christian     0.5245    0.8010    0.6339       387
     rec.motorcycles     0.9403    0.3214    0.4791       392
comp.sys.ibm.pc.hardware     0.5000    0.0052    0.0102       387
        misc.forsale     0.8287    0.6108    0.7033       388
             sci.med     0.9623    0.5312    0.6846       384
  rec.sport.baseball     0.7622    0.6909    0.7248       385
           sci.crypt     0.4503    0.6518    0.5326       382
           sci.space     0.9820    0.4282    0.5964       383
     sci.electronics     0.5520    0.3603    0.4360       383
comp.os.ms-windows.misc     1.0000    0.0026    0.0052       382
       comp.graphics     0.7683    0.1641    0.2704       384

{'detail': {'acq': {'f1': 0.962962962962963,
   'precision': 0.9676966292134831,
   'recall': 0.9582753824756607,
   'support': 719},
  'alt.atheism': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 313},
  'alum': {'f1': 0.5882352941176471,
   'precision': 0.9090909090909091,
   'recall': 0.43478260869565216,
   'support': 23},
  'barley': {'f1': 0.45454545454545453,
   'precision': 0.625,
   'recall': 0.35714285714285715,
   'support': 14},
  'bop': {'f1': 0.5199999999999999,
   'precision': 0.65,
   'recall': 0.43333333333333335,
   'support': 30},
  'carcass': {'f1': 0.6250000000000001,
   'precision': 0.7142857142857143,
   'recall': 0.5555555555555556,
   'support': 18},
  'castor-oil': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
  'cocoa': {'f1': 0.9714285714285714,
   'precision': 1.0,
   'recall': 0.9444444444444444,
   'support': 18},
  'coconut': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 2},
  'coconut-oil': {'f1': 0.0, 'precision': 0.0,

In [ ]:
all_dataframes_test['title'][10360]

['money-fx']

In [ ]:
nltk_tokens = nltk.word_tokenize(text)


In [ ]:
x=all_dataframes_test['text'][10360]


In [ ]:
text=all_dataframes_test['text'][10001]

In [ ]:
text

'jepson corp qtr net shr ct v ct net v sale mln v mln avg shrs mln v mln nine mths shr ct v ct net mln v mln sale mln v mln avg shrs mln v mln qtr per share reflects issuance of two mln share in initial public nine mth result include extraordinary gain of dlrs or ct'

In [ ]:
all_dataframes_train.to_csv("train_allDatasets.csv")
all_dataframes_test.to_csv("test_allDatasets.csv")


In [ ]:
!cp '/content/train_allDatasets.csv' '/content/drive/My Drive/'
!cp '/content/test_allDatasets.csv' '/content/drive/My Drive/'